In [73]:
import pandas as pd

#if we dont skip the first row, we will end up with a dataset with only one column.
data=pd.read_csv('LoanStats3a.csv',low_memory=False,skiprows=1)
print(data.shape)

#pretty interesting size! let's see how can be decrease the size for removing the text columns, such as url
data=data.drop(['url','desc'],axis=1)

# I also remove the columns with more than 50% missing values:
threshold=data.shape[0]/2
data=data.dropna(thresh=threshold,axis=1)
print(data.shape)

#we will save the data in a seprate file, just in case we wanted to epxplore the original file later on.
data.to_csv('loans_2007.csv', index=False)

(42538, 145)
(42538, 53)


Now it's time to take a deeper look at the data dictionary and find the columns that we'd like to drop. The strategy is as follows. We will drop columns that:
* leak information from the future (after the loan has already been funded)
* don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
* formatted poorly and need to be cleaned up
* require more data or a lot of processing to turn into a useful feature
* contain redundant information

For this purpose, I have devided the columns in 3 sets, and then have looked at the data dictionary. The list of columns that will be droped are as follows:

In [74]:
drop_cols=["funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d",
           "zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp",
          "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"]


data=data.drop(drop_cols,axis=1)
data.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',
       'last_credit_pull_d', 'collections_12_mths_ex_med', 'policy_code',
       'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths',
       'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens', 'hardship_flag',
       'disbursement_method', 'debt_settlement_flag'],
      dtype='object')

#### Target variable

The target variable is `loan status`, as we want to see if we can predict the payability of a loan.

In [75]:
target='loan_status'
data[target].value_counts()

Fully Paid                                             34116
Charged Off                                             5670
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

The last two values for target variable do not exist in the data dictionary. Here's when background research comes in handy. Those two values represent the situation that the loan has been approved before, but it does not meet the new requirement. On the hand, we're interested to know whether someone will pay it's loan off or not. In this case, we can drop those values.

In [76]:
data=data[(data[target]=='Fully Paid')|(data[target]=='Charged Off')]

Then I'd like to replace the text values with numbers, I will use the mapping ability of pd.replace() method:

In [77]:
map={
    'loan_status':{
        'Fully Paid':1,
        'Charged Off':0,
    }
}
data=data.replace(map)

In [78]:
data[target].value_counts()

1    34116
0     5670
Name: loan_status, dtype: int64

Now let's find the number of unique values in each column. We'd like to drop any column that has only one unique value, as it will not impact the target variable.

In [79]:
one_unique_cols=[]
for col in data.columns:
    if len(data[col].value_counts().tolist())==1:
        one_unique_cols.append(col)

data=data.drop(one_unique_cols,axis=1)
data.shape

(39786, 24)

#### Preparing the feature and feature selection

My strategy is to first to deal with missing values, then convert categorical features to numerical.

In [80]:
print(data.shape[0])
data.isnull().sum().sort_values(ascending=False)

39786


pub_rec_bankruptcies    697
revol_util               50
title                    10
last_credit_pull_d        2
debt_settlement_flag      0
purpose                   0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
total_acc                 0
loan_amnt                 0
dtype: int64

Missing values are not alot! let's replace the null values with mean of columns:

In [81]:
data=data.fillna(data.mean())
print(data.isnull().sum())

loan_amnt                0
term                     0
int_rate                 0
installment              0
emp_length               0
home_ownership           0
annual_inc               0
verification_status      0
loan_status              0
purpose                  0
title                   10
addr_state               0
dti                      0
delinq_2yrs              0
earliest_cr_line         0
inq_last_6mths           0
open_acc                 0
pub_rec                  0
revol_bal                0
revol_util              50
total_acc                0
last_credit_pull_d       2
pub_rec_bankruptcies     0
debt_settlement_flag     0
dtype: int64


The reason the remaining null values do not change is that those columns are Objects. Let's remove those rows that contain missing values for now.

In [82]:
data=data.dropna(axis=0)

###### dealing with object columns
First let's store them in a seperate dataframe.

In [83]:
object_cols=data.select_dtypes(include=['object'])
object_cols.iloc[0]

term                      36 months
int_rate                     10.65%
emp_length                10+ years
home_ownership                 RENT
verification_status        Verified
purpose                 credit_card
title                      Computer
addr_state                       AZ
earliest_cr_line           Jan-1985
revol_util                    83.7%
last_credit_pull_d         Dec-2017
debt_settlement_flag              N
Name: 0, dtype: object

Let's count the number of unique variables in these columns:

In [84]:
for col in object_cols:
    print('unique values of {0} are'.format(col))
    print(data[col].value_counts())

unique values of term are
 36 months    29041
 60 months    10683
Name: term, dtype: int64
unique values of int_rate are
 10.99%    958
 13.49%    831
 11.49%    824
  7.51%    787
  7.88%    725
  7.49%    656
 11.71%    609
  9.99%    603
  7.90%    582
  5.42%    573
 11.99%    535
 12.69%    492
 10.37%    470
 12.99%    449
  6.03%    447
  8.49%    445
 12.42%    443
 10.65%    435
  5.79%    410
  8.90%    402
  7.29%    397
  6.62%    396
 11.86%    391
 14.27%    389
 10.59%    381
  9.63%    378
  9.91%    377
  5.99%    347
  7.14%    340
  6.99%    336
          ... 
 15.83%      2
 17.15%      2
 15.01%      2
 14.88%      2
 21.82%      2
 20.20%      2
 17.03%      2
 15.07%      2
 14.25%      2
 22.94%      2
 14.62%      2
 18.36%      1
 10.64%      1
 18.72%      1
 16.33%      1
 16.20%      1
 16.01%      1
 17.54%      1
 17.44%      1
 16.15%      1
 21.48%      1
 14.67%      1
 24.59%      1
 22.64%      1
 17.34%      1
 16.96%      1
 24.40%      1
 17.46%  

* Term: 2 values
* int-rate: could be converted to numerical
* emp_lengh: could be converted to numerical
* home_owenership: 4 values
* Purpose and Title have common unique values
* debt_settlement_flag: 2 values
* last_credit_pulled has actually some values in the future (2011 and beyond)

I'll drop the title column, it has some data quality issues including repetitive use of debt consolidation.
Also, considering last_credit_pulled and earliest_cr_line will transform our problem to a time series problem. Therfore, we will drop them for now.

In [85]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

data = data.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
data["int_rate"] = data["int_rate"].str.rstrip("%").astype("float")
data["revol_util"] = data["revol_util"].str.rstrip("%").astype("float")
data=data.replace(mapping_dict)

Let's now encode the rest of categorical columns to dummy variables.

In [86]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term",'debt_settlement_flag']
prefix_dict={
    'home_ownership':'home_own',
    'verification_status':'veri_stat',
    'purpose':'purp',
    'term':'term',
    'debt_settlement_flag':'settle'
}
dummy_df=pd.get_dummies(data[cat_columns],prefix=prefix_dict)
dummy_df.head()
data=pd.concat([data,dumy_df],axis=1)
#dropping the original columns
data=data.drop(cat_columns,axis=1)

#### Model Fitting

First, it is important to notice the class imbalance. Fully paid cases are 6 times more than charged off cases, this might end up overfitting our model.

Also, I plan to use false positive and true positive rates as error metrics. Flase positive indicates "how many percentage of positive outcome does my model mis-predict?" True Positive ratio says: "How many percentage of positive outcome does my model predict right?

We use logistic regression for this binary classification, becuase:
* it's quick to train and we can iterate more quickly,
* it's less prone to overfitting than more complex models like decision trees,
* it's easy to interpret.

In [87]:
features=data.columns.tolist()
features.remove('loan_status')


In [91]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict, KFold

lr = LogisticRegression()
#passing on total number of elements (n), default number of folds are 3.
kf = KFold(data[features].shape[0], random_state=1)

predictions = cross_val_predict(lr, data[features], data[target], cv=kf)
predictions = pd.Series(predictions)

def tpr_fpr(predictions):
    
    # False positives.
    fp_filter = (predictions == 1) & (data["loan_status"] == 0)
    fp = len(predictions[fp_filter])
    
    # True positives.
    tp_filter = (predictions == 1) & (data["loan_status"] == 1)
    tp = len(predictions[tp_filter])
    
    # False negatives.
    fn_filter = (predictions == 0) & (data["loan_status"] == 1)
    fn = len(predictions[fn_filter])
    
    # True negatives
    tn_filter = (predictions == 0) & (data["loan_status"] == 0)
    tn = len(predictions[tn_filter])
    
    # Rates
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    print('tpr is:',tpr)
    print('fpr is:', fpr)

          
tpr_fpr(predictions)

tpr is: 0.9991180880148162
fpr is: 0.9994685562444642


Too much of fpr! we should do something about class imbalance. We have the option of over and under sampling. But I chose to use penalizing the fully paid records.

In [92]:
lr = LogisticRegression(class_weight='balanced')
#passing on total number of elements (n), default number of folds are 3.
kf = KFold(data[features].shape[0], random_state=1)

predictions = cross_val_predict(lr, data[features], data[target], cv=kf)
predictions = pd.Series(predictions)

tpr_fpr(predictions)

tpr is: 0.6285092747743776
fpr is: 0.5918511957484499


Lower fpr, but how about we modify penalizing manually. We can do this by passing a dictionary to weight_class parametr of logictic regression:

In [93]:
penalty = {
    0: 10,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
#passing on total number of elements (n), default number of folds are 3.
kf = KFold(data[features].shape[0], random_state=1)

predictions = cross_val_predict(lr, data[features], data[target], cv=kf)
predictions = pd.Series(predictions)

tpr_fpr(predictions)

tpr is: 0.2034570949819208
fpr is: 0.19238263950398582


Substaintial improvement in fpr, but with the cost of very low tpr. From a conservative investor standpoint, it is good that fpr is low, but we'd like to improve our overall accuracy.

Now,let's use a random forest algorithm, as we have seen nonlinear relationship in our model.

In [99]:
from sklearn.ensemble import RandomForestClassifier

penalty = {
    0: 10,
    1: 1
}

clf=RandomForestClassifier(n_estimators=100, min_samples_leaf=25,class_weight=penalty)
predictions = cross_val_predict(clf, data[features], data[target], cv=kf)
predictions = pd.Series(predictions)

tpr_fpr(predictions)

tpr is: 0.5384072669547579
fpr is: 0.5073516386182463


Obviously, the model is not working properly. There are few ways to approach from here. Ensambling with other algorithms, tuning current algorithms, tweaking penalties, including the columns we already droped, etc.